In [140]:
import pandas as pd
import openpyxl
from gurobipy import *
import matplotlib.pyplot as plt
import numpy as np
import os

In [141]:
all_data_list = os.listdir('/Users/ting/Desktop/testcases-medium')
all_data_list.remove('.DS_Store')
data_num = len(all_data_list)
datadir = '/Users/ting/Desktop/testcases-medium'
instances = []
for i in range(len(all_data_list)):
    name = all_data_list[i]
    fullpath = datadir+'/'+name
    instances.append(pd.read_csv(fullpath,encoding='unicode_escape'))

In [142]:
# job structure 
class Job:
    '''structure for 1 job '''
    def __init__(self, row):
        '''input := df.iloc[idx, :]'''
        self.id = row['Job ID']
        self.due = row['Due Time']
        self.stage_states = [False for _ in range(2)] # True as complete, False as not yet processed
        self.stage_pt = [row['Stage-1 Processing Time'], row['Stage-2 Processing Time']]
        mfor1 = list(map(int, row['Stage-1 Machines'].split(',')))
        if row['Stage-2 Machines'] is not np.nan:
            mfor2 = list(map(int, row['Stage-2 Machines'].split(',')))
        else: mfor2 = [] 
        self.stage_mach = [mfor1, mfor2]
        self.assign_mach = [None for _ in range(2)]
        self.start_time = [-1 for _ in range(2)]
        self.end_time = [-1 for _ in range(2)]
    
    def __repr__(self):
        return f'\
          * Job id: {self.id}\n\
          * Due time:{self.due}\n\
          stage 1: {self.stage_states[0]}, {self.assign_mach[0]}\n\
                   {self.stage_pt[0]}, {self.stage_mach[0]}\n\
          stage 2: {self.stage_states[1]}, {self.assign_mach[1]}\n\
                   {self.stage_pt[1]}, {self.stage_mach[1]}'
    __str__ = __repr__

In [143]:
class Jobs:
    '''structure for multiple jobs' management'''
    def __init__(self, n):
        self.completion_times = np.zeros(n)
        self.tardiness = np.zeros(n)
    def get_RRDD(self):
        if getattr(self, 'RRDD', None) is None:
            self.RRDD = self.due_dates - np.min(self.due_dates)
        return self.RRDD # static
    
    def add_jobs(self, datas):
        self.due_dates = df['Due Time'].to_numpy()
        self.jobs = []
        for i in range(len(datas)):
            row = datas.iloc[i, :]
            jobi = Job(row)
            self.jobs.append(jobi)
    
    def assign(self, job_name, mach, st):
        '''job_name = (2, 0) means job 3 and op 1
        note that job and op is 0-indexed as well as machines
        op
        '''
        
        i = 0 
        jobidx, op = job_name 
        job = self.jobs[jobidx]
        while i < op:
            if job.stage_states[i] is not True:
                print(f'Error scheduling operation: previous operation {i} hasn\'t been scheduled.')
                return 
            i += 1
        J.completion_times[jobidx] = st + job.stage_pt[op]
        job.assign_mach[op] = mach
        job.start_time[op] = st
        job.end_time[op] = J.completion_times[jobidx]
        job.stage_states[op] = True 

In [144]:
class Machines:
    def __init__(self, df):
        '''pass the stage1, stage2 machine lists'''
        mfor1 = df['Stage-1 Machines'].values.tolist()
        mfor2 = df['Stage-2 Machines'].values.tolist()
        mfor1 = [list(map(int, x.split(','))) for x in mfor1]
        mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
        mfor1 = sum(mfor1, [])
        mfor2 = sum(mfor2, [])
        self.number = max(max(mfor1), max(mfor2))
        self.versatile = [mfor1.count(i+1) + mfor2.count(i+1) for i in range(self.number)]
        self.schedule = [[] for _ in range(self.number)]
        self.span = [[] for _ in range(self.number)]
        self.fintime = [0 for _ in range(self.number)]
        
    def is_available(self):
        '''pass machine 編號(1-indexed) to get the state'''
        pass
    def _schedule(self, mach, job_name, proc_time):
        '''mach is 0-indexed'''
        display_name = tuple([x+1 for x in job_name])
        self.schedule[mach].append(display_name) 
        self.span[mach].append(proc_time)
        self.fintime[mach] += proc_time

In [149]:
Total_tardy = []
Total_makespan = []
count = 0
for i in range(data_num):
    df = instances[i]
    due_dates = df['Due Time'].to_numpy()
    M = Machines(df)
    M.versatile
    J = Jobs(len(df))
    J.add_jobs(df)
    # for the first operation
    temp_completion_times = np.array([job.stage_pt[0] for i, job in enumerate(J.jobs)])
    temp_completion_times

    order = np.where(temp_completion_times)# gives stable sort
    # the job index order (0-indexed) to be assigned 
    #np.random.shuffle(order[0])
    # schedule the first operation in accordance to tardiness

    # which index
    batch_index = 0

    for jidx in order[0]:
        job = J.jobs[jidx]
        job_name = (jidx, batch_index)
        job_proc_time = job.stage_pt[batch_index]
        if job_proc_time <= 0:
            # update job
            J.assign(job_name = job_name, 
                    mach = -1,
                    st = job.end_time[batch_index-1]) 
            # note that it's only possible for second operation to have proc time = 0
            # so this doesn't trigger index error
            continue
        # note that the available machines here is 1-indexed, change them to 0-indexed
        avail_machines_idx = [x-1 for x in job.stage_mach[batch_index]]
        # The less versatile, the less fintime, the better. Break ties with smallest-index rule.
        curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x],M.versatile[x], x))
        # schedule the operation on curr_machine
        M._schedule(job_name = job_name, 
                   mach = curr_machine, 
                   proc_time = job_proc_time)
        # update the Jobs objects
        # start time is either concatenated to the machines's last scheduled operation's end
        # or the end of its previous operation 
        J.assign(job_name = job_name, 
                mach = curr_machine, 
                 st = max(M.fintime[curr_machine], J.completion_times[jidx])
                )
    
    batch_index = 1
    for jidx in order[0]:
        job = J.jobs[jidx]
        job_name = (jidx, batch_index)
        job_proc_time = job.stage_pt[batch_index]
        if job_proc_time <= 0:
            # update job
            J.assign(job_name = job_name, 
                    mach = -1,
                    st = job.end_time[batch_index-1]) 
            # note that it's only possible for second operation to have proc time = 0
            # so this doesn't trigger index error
            continue
        # note that the available machines here is 1-indexed, change them to 0-indexed
        avail_machines_idx = [x-1 for x in job.stage_mach[batch_index]]
        # The less versatile, the less fintime, the better. Break ties with smallest-index rule.
        curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x],M.versatile[x], x))
        # schedule the operation on curr_machine
        M._schedule(job_name = job_name, 
                   mach = curr_machine, 
                   proc_time = job_proc_time)
        # update the Jobs objects
        # start time is either concatenated to the machines's last scheduled operation's end
        # or the end of its previous operation 
        J.assign(job_name = job_name, 
                mach = curr_machine, 
                 st = max(M.fintime[curr_machine], J.completion_times[jidx])
                )
    M_span = M.span
    emptylist = []
    Tardy = False
    for i in range(len(M_span)):
        if M_span[i] == emptylist:
            Tardy = True
            break
    if Tardy == True:
        Total_tardy.append(0)
        Total_makespan.append(max(M.fintime))
    else:
        M_complete0 = []
        for i in range(len(M_span)):
                M_complete0.append([M_span[i][0]])
        for k in range(len(M_complete0)):
            for j in range(len(M_span[k])-1):
                M_complete0[k].append(round(sum(M_span[k][0:j+2]),2))


        M_complete = []
        M_start = []
        Machine_ID =[]
        Machine_ID1 =[]
        for i in range(len(order[0])*2):
            M_complete.append(0)
            M_start.append(0)
            Machine_ID.append(0)
            Machine_ID1.append(0)
        for i in range(len(M_complete0)):
                for j in range(len(M_complete0[i])):
                    index1 = M.schedule[i][j][0]
                    index2 = M.schedule[i][j][1]
                    if index2 == 1:
                        M_complete[index1-1] = M_complete0[i][j]
                    else:
                        M_complete[index1-1+len(order[0])] = M_complete0[i][j]
        for i in range(len(M_complete0)):
            for j in range(len(M_complete0[i])-1):
                index1 = M.schedule[i][j+1][0]
                index2 = M.schedule[i][j+1][1]
                if index2 == 1:
                    M_start[index1-1] =  M_complete0[i][j]
                else:
                    M_start[index1-1+len(order[0])] = M_complete0[i][j]

        for i in range(len(M_complete0)):
            for j in range(len(M_complete0[i])):
                index1 = M.schedule[i][j][0]
                index2 = M.schedule[i][j][1]
                if index2 == 1:
                    Machine_ID[index1-1] = i+1
                    Machine_ID1[index1-1] = i+1
                else:
                    Machine_ID[index1-1+len(order[0])] = i+1
                    Machine_ID1[index1-1+len(order[0])] = i+1

        Machine_array = np.array(Machine_ID)
        Machine_index = np.where(Machine_array == 0)
        Machine_index_list = list(Machine_index[0]-len(order[0]))
        Machine_index_list0 = list(Machine_index[0])

        for i in range(len(Machine_index_list)):
            Machine_ID[Machine_index_list[i]] = 0
        for i in range(len(Machine_index_list0)):
            Machine_ID1[Machine_index_list0[i]] = None

        Completion_time_Job = []
        for j in range(len(order[0])):
            Completion_time_Job.append(max(M_complete[j],M_complete[j+len(order[0])]))
        Completion_times = np.array(Completion_time_Job)

        Tardy_number = len(np.where(Completion_times > J.due_dates)[0])
        Total_tardy.append(Tardy_number)
        Makespan = max(M.fintime)
        Total_makespan.append(Makespan)
print(Total_tardy)
average_tardy = sum(Total_tardy)/data_num
average_makespan = sum(Total_makespan)/data_num
print('Random MEDIUM cases (147 instances):')
print("average tardy job for all these cases "+str(average_tardy))
print("average makespans for all these cases "+str(average_makespan))

[22, 2, 2, 15, 21, 6, 42, 16, 3, 11, 2, 11, 10, 0, 2, 0, 35, 37, 2, 9, 2, 49, 38, 3, 0, 2, 28, 6, 21, 27, 29, 3, 11, 12, 2, 19, 11, 12, 48, 0, 2, 36, 0, 29, 0, 17, 9, 18, 27, 48, 2, 22, 0, 0, 49, 31, 6, 0, 3, 32, 41, 0, 2, 30, 27, 22, 51, 49, 32, 48, 0, 0, 9, 8, 2, 0, 14, 2, 31, 10, 25, 5, 6, 37, 12, 17, 5, 0, 16, 14, 14, 10, 22, 33, 0, 4, 1, 0, 0, 8, 46, 3, 0, 6, 3, 28, 5, 2, 0, 3, 4, 0, 1, 0, 0, 4, 37, 0, 11, 18, 0, 1, 0, 3, 0, 9, 9, 1, 7, 3, 1, 40, 11, 0, 31, 6, 1, 42, 0, 2, 2, 5, 1, 2, 10, 24, 4]
Random MEDIUM cases (147 instances):
average tardy job for all these cases 12.891156462585034
average makespans for all these cases 21.85714285714287


In [146]:
FCFS_tardy_ave = 97.95238095238095
FCFS_makespans_ave = 231.22108843537364
Random_tardy_ave = 100.15646258503402
Random_makespans_ave = 231.80136054421732
Heu_tardy_ave = 10.918367346938776
Heu_makespans_ave = 25.73741496598644

In [147]:
print('FCFS MEDIUM cases (147 instances):')
print("average tardy job for all these cases: "+str(FCFS_tardy_ave))
print("average makespans for all these cases: "+str(FCFS_makespans_ave))
print('Random MEDIUM cases (147 instances):')
print("average tardy job for all these cases: "+str(Random_tardy_ave))
print("average makespans for all these cases: "+str(Random_makespans_ave))
print("Average tardy job ratio of FCFS to Random:"+str(FCFS_tardy_ave/Random_tardy_ave))
print("Average makespans ratio of FCFS to Random:"+str(FCFS_makespans_ave/Random_makespans_ave))
print("Average tardy job ratio of Heuristic to FCFS:"+str(Heu_tardy_ave/FCFS_tardy_ave))
print("Average makespans ratio of Heuristic to FCFS:"+str(Heu_makespans_ave/FCFS_makespans_ave))

FCFS MEDIUM cases (147 instances):
average tardy job for all these cases: 97.95238095238095
average makespans for all these cases: 231.22108843537364
Random MEDIUM cases (147 instances):
average tardy job for all these cases: 100.15646258503402
average makespans for all these cases: 231.80136054421732
Average tardy job ratio of FCFS to Random:0.9779936154316374
Average makespans ratio of FCFS to Random:0.9974966837663017
Average tardy job ratio of Heuristic to FCFS:0.11146607403291897
Average makespans ratio of Heuristic to FCFS:0.11131084599655819
